In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['author'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,ridge,shock,presence,vegetable,courtier,light,depend,dismay,reformation,hall,...,irresistible,today,laughing,spirit,connection,rude,fail,execute,text_sentence,author
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Instantiate our model and fit the data.

svm = SVC(kernel = 'linear')
Y = word_counts['author']
X = np.array(word_counts.drop(['text_sentence','author'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
test = svm.fit(X_test, y_test)
print(X_test.shape, y_test.shape)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

(2128, 3062) (2128,)
Training set score: 0.889655172414

Test set score: 0.978853383459


In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(test, X_test, y_test, cv=10)

array([ 0.92523364,  0.90186916,  0.90140845,  0.89201878,  0.88732394,
        0.87323944,  0.89622642,  0.89622642,  0.87735849,  0.91509434])

**The cross validation scores suggest overfitting, as do the scores comparing testing data to training data.  Providing the model with more data should help combat overfitting.**

In [ ]:
# count punctuation by sentence and concat 

In [ ]:
# words per sentence

In [9]:
# function to count punctuation

def punct_frequencies(text):

    punct = []

    for token in text:
        if token.is_punct:
            punct.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(punct)

alice_punct = punct_frequencies(alice_doc)
alice_punct['text'] = 1
persuasion_punct = punct_frequencies(persuasion_doc)
persuasion_punct['text'] = 0

In [10]:
punct = (alice_punct + persuasion_punct)

In [27]:
ali = pd.DataFrame(alice_punct, index=range(0,16))

#df.columns[0] = 'ID' 
#df['ID'] = df.index + 0

In [30]:
ali.head(10)

,!,"""",',(,),*,",",-,.,:,;,?,[,],_,text
0,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
1,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
2,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
3,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
4,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
5,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
6,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
7,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
8,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1
9,450,113,2220,56,56,60,2418,137,976,233,194,201,2,2,4,1


In [11]:
punct

Counter({'!': 768,
         '"': 1677,
         '&': 4,
         "'": 2265,
         '(': 126,
         ')': 126,
         '*': 60,
         ',': 9442,
         '-': 445,
         '.': 4096,
         ':': 363,
         ';': 1513,
         '?': 418,
         '[': 2,
         ']': 2,
         '_': 4,
         'text': 1})

In [12]:
punct_count = pd.DataFrame(alice_punct, persuasion_punct, punct)
punct_count.head()

,",",:,',?,(,),-,.,;,!,"""",*,_,[,],text,&
",",2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
;,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
.,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
:,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
"""",2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN


In [14]:
punct_count[punct_count['text'] == 1]

,",",:,',?,(,),-,.,;,!,"""",*,_,[,],text,&
",",2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
;,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
.,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
:,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
"""",2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
(,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
),2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
-,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
',2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN
?,2418,233,2220,201,56,56,137,976,194,450,113,60,4,2,2,1,NaN


In [17]:
punct_count = pd.DataFrame(columns=['alice', 'persuasion'])
punct_count.head()

,alice,persuasion


In [ ]:
#punct_count['alice'] = np.where(punct_count['alice'].isin([alice_doc])), 1, 0
#punct_count['persuasion'] = np.where(punct_count['persuasion'].isin([persuasion_doc])), 1, 0

In [ ]:
#np.where(punct_count['alice'].isin([alice_doc])), 1, 0

In [22]:
for token in alice_doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Alice alice PROPN NNP nsubj Xxxxx True False
was be VERB VBD aux xxx True True
beginning begin VERB VBG ROOT xxxx True False
to to PART TO aux xx True True
get get VERB VB xcomp xxx True True
very very ADV RB advmod xxxx True True
tired tired ADJ JJ acomp xxxx True False
of of ADP IN prep xx True True
sitting sit VERB VBG pcomp xxxx True False
by by ADP IN prep xx True True
her -PRON- ADJ PRP$ poss xxx True True
sister sister NOUN NN pobj xxxx True False
on on ADP IN prep xx True True
the the DET DT det xxx True True
bank bank NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
of of ADP IN conj xx True True
having have VERB VBG pcomp xxxx True False
nothing nothing NOUN NN dobj xxxx True True
to to PART TO aux xx True True
do do VERB VB relcl xx True True
: : PUNCT : punct : False False
once once ADV RB advmod xxxx True True
or or CCONJ CC cc xx True True
twice twice ADV RB conj xxxx True False
she -PRON- PRON PRP nsubj xxx True True
had have

for for ADP IN prep xxx True True
some some DET DT det xxxx True True
way way NOUN NN pobj xxx True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
then then ADV RB advmod xxxx True True
dipped dip VERB VBD conj xxxx True False
suddenly suddenly ADV RB advmod xxxx True False
down down ADV RB advmod xxxx True True
, , PUNCT , punct , False False
so so ADV RB advmod xx True True
suddenly suddenly ADV RB advmod xxxx True False
that that ADP IN mark xxxx True True
Alice alice PROPN NNP nsubj Xxxxx True False
had have VERB VBD ccomp xxx True True
not not ADV RB neg xxx True True
a a DET DT det x True True
moment moment NOUN NN dobj xxxx True False
to to PART TO aux xx True True
think think VERB VB relcl xxxx True False
about about ADP IN prep xxxx True True
stopping stop VERB VBG pcomp xxxx True False
herself -PRON- PRON PRP dobj xxxx True True
before before ADP IN mark xxxx True True
she -PRON- PRON PRP nsubj xxx True True
found find VERB VBD advcl xxxx True False
h

I -PRON- PRON PRP nsubj X True False
would would VERB MD aux xxxx True True
n't not ADV RB neg x'x False False
say say VERB VB ROOT xxx True True
anything anything NOUN NN dobj xxxx True True
about about ADP IN prep xxxx True True
it -PRON- PRON PRP pobj xx True True
, , PUNCT , punct , False False
even even ADV RB advmod xxxx True True
if if ADP IN mark xx True True
I -PRON- PRON PRP nsubj X True False
fell fall VERB VBD advcl xxxx True False
off off ADP IN prep xxx True True
the the DET DT det xxx True True
top top NOUN NN pobj xxx True True
of of ADP IN prep xx True True
the the DET DT det xxx True True
house house NOUN NN pobj xxxx True False
! ! PUNCT . punct ! False False
' ' PUNCT '' punct ' False False
( ( PUNCT -LRB- punct ( False False
Which which ADJ WDT nsubj Xxxxx True False
was be VERB VBD ROOT xxx True True
very very ADV RB advmod xxxx True True
likely likely ADV RB acomp xxxx True False
true true ADJ JJ acomp xxxx True False
. . PUNCT . punct . False False
) ) PUNCT -RR

it -PRON- PRON PRP nsubj xx True True
did do VERB VBD aux xxx True True
n't not ADV RB neg x'x False False
sound sound VERB VB advcl xxxx True False
at at ADV RB advmod xx True True
all all ADJ PDT predet xxx True True
the the DET DT det xxx True True
right right ADJ JJ amod xxxx True False
word word NOUN NN dobj xxxx True False
) ) PUNCT -RRB- punct ) False False
' ' PUNCT '' punct ' False False
but but CCONJ CC cc xxx True True
I -PRON- PRON PRP nsubj X True False
shall shall VERB MD aux xxxx True False
have have VERB VB ROOT xxxx True True
to to PART TO aux xx True True
ask ask VERB VB xcomp xxx True False
them -PRON- PRON PRP dobj xxxx True True
what what NOUN WP attr xxxx True True
the the DET DT det xxx True True
name name NOUN NN nsubj xxxx True True
of of ADP IN prep xx True True
the the DET DT det xxx True True
country country NOUN NN pobj xxxx True False
is be VERB VBZ ccomp xx True True
, , PUNCT , punct , False False
you -PRON- PRON PRP nsubj xxx True True
know know VERB VB

? ? PUNCT . punct ? False False
Do do VERB VBP aux Xx True False
cats cat NOUN NNS nsubj xxxx True False
eat eat VERB VB ROOT xxx True False
bats bat NOUN NNS dobj xxxx True False
? ? PUNCT . punct ? False False
' ' PUNCT '' punct ' False False
and and CCONJ CC cc xxx True True
sometimes sometimes ADV RB advmod xxxx True True
, , PUNCT , punct , False False
' ' PUNCT '' punct ' False False
Do do VERB VB aux Xx True False
bats bat NOUN NNS nsubj xxxx True False
eat eat VERB VB ROOT xxx True False
cats cat NOUN NNS dobj xxxx True False
? ? PUNCT . punct ? False False
' ' PUNCT '' punct ' False False
for for ADP IN ROOT xxx True True
, , PUNCT , punct , False False
you -PRON- PRON PRP nsubj xxx True True
see see VERB VBP parataxis xxx True True
, , PUNCT , punct , False False
as as ADP IN mark xx True True
she -PRON- PRON PRP nsubj xxx True True
could could VERB MD aux xxxx True True
n't not ADV RB neg x'x False False
answer answer VERB VB advcl xxxx True False
either either DET DT det xx

...

yourself -PRON- PRON PRP pobj xxxx True True
and and CCONJ CC cc xxx True True
me -PRON- PRON PRP conj xx True True
. . PUNCT . punct . False False
' ' PUNCT '' punct ' False False
' ' PUNCT '' punct ' False False
That that DET DT nsubj Xxxx True False
's be VERB VBZ ccomp 'x False False
the the DET DT det xxx True True
most most ADV RBS advmod xxxx True True
important important ADJ JJ amod xxxx True False
piece piece NOUN NN attr xxxx True False
of of ADP IN prep xx True True
evidence evidence NOUN NN pobj xxxx True False
we -PRON- PRON PRP nsubj xx True True
've have VERB VB aux 'xx False False
heard hear VERB VBN relcl xxxx True False
yet yet ADV RB advmod xxx True True
, , PUNCT , punct , False False
' ' PUNCT '' punct ' False False
said say VERB VBD ccomp xxxx True False
the the DET DT det xxx True True
King king PROPN NNP nsubj Xxxx True False
, , PUNCT , punct , False False
rubbing rub VERB VBG advcl xxxx True False
his -PRON- ADJ PRP$ poss xxx True True
hands hand NOUN NNS dobj

he -PRON- PRON PRP nsubj xx True True
found find VERB VBD advcl xxxx True False
it -PRON- PRON PRP nsubj xx True True
made make VERB VBD ccomp xxxx True True
no no DET DT det xx True True
mark mark NOUN NN dobj xxxx True False
; ; PUNCT : punct ; False False
but but CCONJ CC cc xxx True True
he -PRON- PRON PRP nsubj xx True True
now now ADV RB advmod xxx True True
hastily hastily ADV RB advmod xxxx True False
began begin VERB VBD conj xxxx True False
again again ADV RB advmod xxxx True True
, , PUNCT , punct , False False
using use VERB VBG advcl xxxx True True
the the DET DT det xxx True True
ink ink NOUN NN dobj xxx True False
, , PUNCT , punct , False False
that that ADJ WDT nsubj xxxx True True
was be VERB VBD aux xxx True True
trickling trickle VERB VBG conj xxxx True False
down down PART RP prt xxxx True True
his -PRON- ADJ PRP$ poss xxx True True
face face NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
as as ADV RB advmod xx True True
long long ADV RB advmod xxxx T

them -PRON- PRON PRP dobj xxxx True True
off off PART RP prt xxx True True
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
found find VERB VBD conj xxxx True False
herself -PRON- PRON PRP nsubj xxxx True True
lying lie VERB VBG ccomp xxxx True False
on on ADP IN prep xx True True
the the DET DT det xxx True True
bank bank NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
with with ADP IN prep xxxx True True
her -PRON- ADJ PRP$ poss xxx True True
head head NOUN NN pobj xxxx True False
in in ADP IN prep xx True True
the the DET DT det xxx True True
lap lap NOUN NN pobj xxx True False
of of ADP IN prep xx True True
her -PRON- ADJ PRP$ poss xxx True True
sister sister NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
who who NOUN WP nsubj xxx True True
was be VERB VBD aux xxx True True
gently gently ADV RB advmod xxxx True False
brushing brush VERB VBG relcl xxxx True False
away away PART RP advmod xxxx True False
some some DET DT det xxxx True True


once once ADV RB advmod xxxx True True
again again ADV RB advmod xxxx True True
the the DET DT det xxx True True
tiny tiny ADJ JJ amod xxxx True False
hands hand NOUN NNS nsubjpass xxxx True False
were be VERB VBD auxpass xxxx True True
clasped clasp VERB VBN conj xxxx True False
upon upon ADP IN prep xxxx True True
her -PRON- ADJ PRP$ poss xxx True True
knee knee NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
the the DET DT det xxx True True
bright bright ADJ JJ advmod xxxx True False
eager eager ADJ JJ amod xxxx True False
eyes eye NOUN NNS nsubj xxxx True False
were be VERB VBD aux xxxx True True
looking look VERB VBG conj xxxx True False
up up PART RP prt xx True True
into into ADP IN prep xxxx True True
hers -PRON- PRON PRP pobj xxxx True True
she -PRON- PRON PRP nsubj xxx True True
could could VERB MD aux xxxx True True
hear hear VERB VB conj xxxx True False
the the DET DT det xxx True True
very very ADJ JJ amod xxxx True True
tones

feel feel VERB VB conj xxxx True False
with with ADP IN prep xxxx True True
all all ADJ PDT det xxx True True
their -PRON- ADJ PRP$ poss xxxx True True
simple simple ADJ JJ amod xxxx True False
sorrows sorrow NOUN NNS pobj xxxx True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
find find VERB VB conj xxxx True False
a a DET DT det x True True
pleasure pleasure NOUN NN dobj xxxx True False
in in ADP IN prep xx True True
all all ADJ PDT predet xxx True True
their -PRON- ADJ PRP$ poss xxxx True True
simple simple ADJ JJ amod xxxx True False
joys joy NOUN NNS pobj xxxx True False
, , PUNCT , punct , False False
remembering remember VERB VBG conj xxxx True False
her -PRON- ADJ PRP$ poss xxx True True
own own ADJ JJ amod xxx True True
child child NOUN NN compound xxxx True False
- - PUNCT HYPH punct - False False
life life NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
and and CCONJ CC cc xxx True True
the the DET DT det xxx True True
happy happy ADJ J

In [ ]:
text = alice_doc

punct = []

for token in text:
        if token.is_punct:
            punct.append(token.text)
            
    # Build and return a Counter object containing word counts.
print(Counter(punct))

In [43]:
words_punct = pd.concat([word_counts, alice_punct, persuasion_punct], 
                       axis =1)

TypeError: cannot concatenate a non-NDFrame object

In [ ]:
df2 = pd.concat([word_counts, punct_frequencies], axis=1)

In [30]:
punct_symbol = set(alicepunct + persuasionpunct)

In [ ]:
df2.head()

In [ ]:
df2 = pd.concat([word_counts, punct_frequencies], axis=1)

In [ ]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words), punct_frequencies)
word_counts.head()

In [ ]:
for token in sents:
    print(token.orth_, token.dep_, token.head.orth_)